# Missing Values - Valores Ausentes

## Class

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [3]:
# Load the data
data = pd.read_csv('CSV/melb_data.csv')

# Select target
y = data.Price

# To keep things simple, we'll use only numerical predictors
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)


### score_dataset() - function to compare different missing values

In [4]:
def score_dataset(X_train, X_valid, y_train, y_valid):
    # n_estimators = representa o número de ARVORES *na* FLORESTA (ensemble) que o modelo irá construir.
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### First - Drop Columns with Missing Values - Eliminar colunas com valores ausentes.

In [5]:
#cols_with_missing = [col for col in X_train.columns
#                    if X_train[col].isnull().any()]

cols_with_missing = []
for col in X_train.columns:
    if X_train[col].isnull().any():
        cols_with_missing.append(col)


#axis=1 - colunas
#axis=0 - linhas


reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with missing values):
183550.22137772635


### Second - Inmputation - Imputação

In [6]:
from sklearn.impute import SimpleImputer

# Imputação 
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.fit_transform(X_valid))

# Imputação removeu nomes de colunas;   coloque-os de volta 
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns


print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
179816.89508731329


### Third - Extension to Imputation - Extensão da Amputação

In [12]:
from sklearn.impute import SimpleImputer

# cópia para evitar alterar os dados originais (ao imputar)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

cols_with_missing = []
for col in X_train.columns:
    if X_train[col].isnull().any():
        cols_with_missing.append(col)


for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()


my_imputer = SimpleImputer()

#fit_transform - usado para dodos treinamento
#transform - usado para dados teste
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))


# Imputação removeu nomes de colunas;  coloque-os de volta
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
178927.503183954


In [16]:
# Forma dos dados de treinamento (num_rows, num_columns)
#print(X_train.shape)
print(X_train)


# Número de valores ausentes em cada coluna de dados de treinamento 
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])



       Rooms  Distance  Postcode  Bedroom2  Bathroom  Car  Landsize  \
12167      1       5.0    3182.0       1.0       1.0  1.0       0.0   
6524       2       8.0    3016.0       2.0       2.0  1.0     193.0   
8413       3      12.6    3020.0       3.0       1.0  1.0     555.0   
2919       3      13.0    3046.0       3.0       1.0  1.0     265.0   
6043       3      13.3    3020.0       3.0       1.0  2.0     673.0   
...      ...       ...       ...       ...       ...  ...       ...   
13123      3       5.2    3056.0       3.0       1.0  2.0     212.0   
3264       3      10.5    3081.0       3.0       1.0  1.0     748.0   
9845       4       6.7    3058.0       4.0       2.0  2.0     441.0   
10799      3      12.0    3073.0       3.0       1.0  1.0     606.0   
2732       4       6.4    3011.0       4.0       2.0  1.0     319.0   

       BuildingArea  YearBuilt  Lattitude  Longtitude  Propertycount  
12167           NaN     1940.0  -37.85984   144.98670        13240.0  
6524 

## Exercise: Missing Values

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Ler arquivos
X_full = pd.read_csv('CSV/home-data-for-ml-course/train.csv', index_col='Id')
X_test_full = pd.read_csv('CSV/home-data-for-ml-course/test.csv', index_col='Id')

# Remova linhas com destino ausente, separe o alvo dos preditores
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# Para simplificar, usaremos apenas preditores numéricos
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [2]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


In [3]:
# Formato dos dados de treinamento (num_linhas, num_colunas)
print(X_train.shape)

# numero de valores ausentes em cada coluna de dados de treinamento
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [4]:
# Quantas linhas existem nos dados de treinamento?
num_rows = X_train.shape[0]
print(num_rows)

# Quantas colunas nos dados de treinamento tem valores ausentes?
num_cols_with_missing = len(missing_val_count_by_column[missing_val_count_by_column > 0])
print(num_cols_with_missing)

# Quantas entradas faltantes tem nos dados de treinamento?
tot_missing = missing_val_count_by_column.sum()
print(tot_missing)

1168
3
276


### score_dataset()

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### Drop Columns With Missing Values - Elliminar Colunas Com Valores Ausentes

In [6]:
# Obter nome das colunas com valores faltantes
cols_with_missing = []
for col in X_train.columns:
    if X_train[col].isnull().any():
        cols_with_missing.append(col)

# Remover do treinamento e da validação essas colunas com valores faltantes
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

In [7]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
17837.82570776256


### Imputation - Imputação

In [8]:
from sklearn.impute import SimpleImputer

# Fill in the lines below: imputation
my_imputer = SimpleImputer()

# Gera novas colunas, que diz o que será amputado
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns


In [9]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
18062.894611872147


### Applying Any Type of Resolution to Missing Values - Aplicação de Qualquer Tipo de Resolução Para Valores Ausentes

In [10]:
# median fez diferença em relação ao que foi aplicado anteriormente

final_imputer = SimpleImputer(strategy='median') 

final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))

# Imputação removeu nomes de colunas;  coloque-os de volta
final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns

In [11]:
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

preds_valid = model.predict(final_X_valid)

print("(MAE) - Minha abordagem: ")
print(mean_absolute_error(y_valid, preds_valid))

(MAE) - Minha abordagem: 
17791.59899543379


In [12]:
final_X_test = pd.DataFrame(final_imputer.transform(X_test))

# Get test predictions
preds_test = model.predict(final_X_test)
preds_test = pd.DataFrame(preds_test)
preds_test

/home/haise/miniconda3/envs/data/lib/python3.12/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


,0
0,125985.50
1,154599.50
2,180070.24
3,183544.50
4,197549.92
...,...
1454,87437.12
1455,86994.50
1456,154751.44
1457,107387.00


In [ ]:
# Salvar test predictions em arquivo
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)